In [1]:
import numpy as np
import pandas as pd

Data cleaning is definitely a "practice makes perfect" skill. Using this dataset of article open-access prices paid by the WELLCOME Trust between 2012 and 2013, determine the five most common journals and the total articles for each. Next, calculate the mean, median, and standard deviation of the open-access cost per article for each journal . You will need to do considerable data cleaning in order to extract accurate estimates. For a real bonus round, identify the open access prices paid by subject area.

In [2]:
wellcome = pd.read_csv('WELLCOME_APCspend2013_forThinkful.csv', encoding='latin1')
print(wellcome.dtypes)
wellcome.columns = ['id', 'publisher', 'journal', 'title', 'cost']
wellcome.head()

PMID/PMCID                                             object
Publisher                                              object
Journal title                                          object
Article title                                          object
COST (£) charged to Wellcome (inc VAT when charged)    object
dtype: object


,id,publisher,journal,title,cost
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88


In [3]:
wellcome.describe()

,id,publisher,journal,title,cost
count,1928,2127,2126,2127,2127
unique,1880,299,984,2126,1402
top,Not yet available,Elsevier,PLoS One,"Exclusive breastfeeding, diarrhoel morbidity a...",£2040.00
freq,7,387,92,2,94


In [4]:
# Ensure every element in the DataFrame is a string
wellcome = wellcome.applymap(str)

# Remove leading and trailing whitespace
wellcome = wellcome.applymap(lambda item: item.strip())

# Filter out records which have a float64 type NaN as an ID
wellcome = wellcome[wellcome.id.map(lambda item: item != 'nan')]

# Remove spaces in ID fields
wellcome.id = wellcome.id.map(lambda item: item.replace(' ', ''))

# Remove currency sign from price variable
wellcome.cost = wellcome.cost.map(lambda item: item.replace('£', ''))
wellcome.cost = wellcome.cost.map(lambda item: item.replace('$', ''))

# Convert cost variable to Pandas float type
wellcome.cost = wellcome.cost.astype(dtype='float64')

# Top 5 journals
# wellcome.groupby('journal').nlargest

In [5]:
wellcome.describe(include='all')

,id,publisher,journal,title,cost
count,1928,1928,1928,1928,1928.000000
unique,1877,251,874,1928,NaN
top,Notyetavailable,Elsevier,PLoS One,A role for piezo2 in EPAC1 dependent mechanic...,NaN
freq,7,352,91,1,NaN
mean,NaN,NaN,NaN,NaN,22729.969891
std,NaN,NaN,NaN,NaN,142424.454810
min,NaN,NaN,NaN,NaN,9.930000
25%,NaN,NaN,NaN,NaN,1280.000000
50%,NaN,NaN,NaN,NaN,1868.810000
75%,NaN,NaN,NaN,NaN,2315.555000


In [6]:
help(wellcome.groupby('journal').nunique)

Help on method nunique in module pandas.core.groupby:

nunique(dropna=True) method of pandas.core.groupby.DataFrameGroupBy instance
    Return DataFrame with number of distinct observations per group for
    each column.
    
    .. versionadded:: 0.20.0
    
    Parameters
    ----------
    dropna : boolean, default True
        Don't include NaN in the counts.
    
    Returns
    -------
    nunique: DataFrame
    
    Examples
    --------
    >>> df = pd.DataFrame({'id': ['spam', 'egg', 'egg', 'spam',
    ...                           'ham', 'ham'],
    ...                    'value1': [1, 5, 5, 2, 5, 5],
    ...                    'value2': list('abbaxy')})
    >>> df
         id  value1 value2
    0  spam       1      a
    1   egg       5      b
    2   egg       5      b
    3  spam       2      a
    4   ham       5      x
    5   ham       5      y
    
    >>> df.groupby('id').nunique()
        id  value1  value2
    id
    egg    1       1       1
    ham    1       1    

In [7]:
# Remove known special characters
wellcome.id = wellcome.id.map(lambda item: item.replace(':', ''))
wellcome.id = wellcome.id.map(lambda item: item.replace('(', ''))
wellcome.id = wellcome.id.map(lambda item: item.replace(')', ''))
wellcome.id = wellcome.id.map(lambda item: item.replace(',', ''))
wellcome.id = wellcome.id.map(lambda item: item.replace('-', ''))
wellcome.id = wellcome.id.map(lambda item: item.replace('\n', ''))
wellcome.id = wellcome.id.map(lambda item: item.replace('[', ''))
wellcome.id = wellcome.id.map(lambda item: item.replace(']', ''))
wellcome.id = wellcome.id.map(lambda item: item.replace('/', ''))
wellcome.id = wellcome.id.map(lambda item: item.replace('.', ''))
wellcome.id = wellcome.id.map(lambda item: item.replace('?', ''))

# Remove known ID A-Z characters
wellcome.id = wellcome.id.map(lambda item: item.replace('PMCID', ''))
wellcome.id = wellcome.id.map(lambda item: item.replace('PMID', ''))
wellcome.id = wellcome.id.map(lambda item: item.replace('PMC', ''))
wellcome.id = wellcome.id.map(lambda item: item.replace('PM', ''))

# Extract digits from every ID entry
wellcome.id = wellcome.id.map(lambda x: ''.join(list(filter(str.isdigit, x))))

# Filter out records whose ID is less than 7 characters - the minimum length of one of these types of ID's
wellcome = wellcome[wellcome.id.map(lambda item: len(item) > 6)]

# Remove ID's which are only letters
# wellcome = wellcome[wellcome.id.map(lambda item: item.isalpha() == False)]

In [8]:
wellcome.describe(include='all')

,id,publisher,journal,title,cost
count,1846,1846,1846,1846,1846.000000
unique,1833,246,835,1846,NaN
top,22735079,Elsevier,PLoS One,A role for piezo2 in EPAC1 dependent mechanic...,NaN
freq,2,336,90,1,NaN
mean,NaN,NaN,NaN,NaN,22030.752974
std,NaN,NaN,NaN,NaN,140051.222774
min,NaN,NaN,NaN,NaN,9.930000
25%,NaN,NaN,NaN,NaN,1268.700000
50%,NaN,NaN,NaN,NaN,1836.920000
75%,NaN,NaN,NaN,NaN,2312.117500


In [9]:
# Remove ID's which are 
# wellcome = wellcome[wellcome.id.map(lambda item: item.isalpha() == False)]
# wellcome = wellcome[wellcome.id.map(lambda item: item.isalnum() == False)]

# Remove ID's which are alphanumeric but do not contain "PM"